

Yelp Review dataset is quite large with a set of 560,000 highly polar yelp reviews for training. The testing data consists of 38,000 reviews. This is a dataset for binary classification. This dataset is a subset of the yelp dataset that was originally published in the Yelp Dataset challenge in 2015. <br> 


The Yelp Review Polarity dataset is constructed by categorizing 1 and 2 stars as negative reviews and 3 and 4 stars as positive reviews. We have 280k train positive and 280 train negative reviews and 19k positive test and 19k negative test reviews. <br>

Homepage: https://course.fast.ai/datasets<br>

Source code: tfds.text.yelp_polarity_reviews

Versions:

0.2.0 (default): No release notes.
Download size: 158.67 MiB

Dataset size: 435.14 MiB

Since 2017, the Transformer architecture model started to feature more prominently in natural language processing. Using neural attention, it is possible to build powerful sequence models without the need for recurrent or convolution layers. Essentially it takes a sequence as input (often a sentence or paragraph) and translates that into a different sequence. This is the task highly influential for many of the important natural language appplications:


*   Text Generation
*   Chatbots
*   Translations
*  Summarization







In short, using self-attention involves scoring the important features of a set of text or an image. Higher score is given to important features and lower scores around the edges or less important features. In natural language processing, self-attention makes word embeddings smarter by representing tokens in the sequence that makes them context-aware. 

## Import Packages

In [ ]:
import re
import string
from packaging import version
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow import keras


import numpy as np
import pandas as pd
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten, Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from tensorflow.keras.layers import TextVectorization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def version_checker():
  """The function makes sure we are using the correct version of keras and tensorflow."""
  print("This notebook requires TensorFlow 2.0 or above")
  print("TensorFlow version: ", tf.__version__)
  assert version.parse(tf.__version__).release[0] >=2
  print("Keras version: ", keras.__version__)
version_checker()

This notebook requires TensorFlow 2.0 or above
TensorFlow version:  2.8.2
Keras version:  2.8.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# register  yelp_polarity_reviews so that tfds.load doesn't generate a checksum (mismatch) error
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums --datasets=yelp_polarity_reviews

dataset, info = tfds.load('yelp_polarity_reviews', with_info=True,  split=['train[:95%]','train[95%:]', 'test'],batch_size = 32, as_supervised=True)
train_ds, val_ds, test_ds = dataset

I0604 16:44:04.886303 140429515790208 download_and_prepare.py:200] Running download_and_prepare for dataset(s):
yelp_polarity_reviews
I0604 16:44:04.886712 140429515790208 dataset_builder.py:811] No config specified, defaulting to first: yelp_polarity_reviews/plain_text
I0604 16:44:04.887211 140429515790208 dataset_info.py:361] Load dataset info from /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0
I0604 16:44:04.889339 140429515790208 dataset_info.py:361] Load dataset info from /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0
I0604 16:44:04.890531 140429515790208 download_and_prepare.py:138] download_and_prepare for dataset yelp_polarity_reviews/plain_text/0.1.0...
I0604 16:44:04.890706 140429515790208 dataset_builder.py:299] Reusing dataset yelp_polarity_reviews (/root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0)
name: "yelp_polarity_reviews"
description: "Large Yelp Review Dataset.\nThis is a dataset for binary sentiment classification.

During the vectorization, I selected the max_token argument to be 300 which means that the maximum size of the vocabulary is 300. \
Secondly, i select the output sequence length to be 3000, which means that the output has time dimension padded to 3000. \
Thirdly, I am outputting integers so the vectorization step outputs integer indices - one integer index per token.\
Finally, I am doing preprocessing by converting all text to lowercase and stripping any punctuations from the tokens.

In [ ]:
max_length = 300
max_tokens = 3000
lower_no_punc = "lower_and_strip_punctuation"
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    standardize =lower_no_punc,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

## Transformer Encoder Implemented As Subclassed `Layer`

In [ ]:
class TransformerEncoder(layers.Layer):
  """Representation of a transformer for sentiment classification purposes."""
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    """Instantiating the Transformer.
    Inputs: 
    embed_dim: size of the input token vectors
    dense_dim: size of the hidden dense layer
    num_heads: number of attention heads"""
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.attention = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"),
          layers.Dense(embed_dim),]
          )


  def call(self, inputs, mask=None):
    """We are defining the forward pass of the model in the call method. This uses the previous layers created in the model.
    Instantiate the subclass and call it on the data to create model weights."""
    if mask is not None:
        mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask)

    return self.layernorm_2(proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
    })

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/TransformerEncoder.png?raw=1">

I have used a vocabulary size of 3000 to make sure we lose less semantics from the text body of each of the reviews.

Secondly, I am only dropping 25% of the lowest performing nodes to retain information from 75% of the nodes.

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(2, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="SparseCategoricalCrossentropy",)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         768000    
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 514   

We are applying two callbacks in the fit method using the callbacks arg. It takes a list of callbacks. First callback is the model checkpoint that save the current weights after every epoch. Second callbacks is the early stopping. It monitors the model's validation accuracy and interrupts training when accuracy has stopped improving for three epochs.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    ]
                                      
model.fit(int_train_ds, validation_data=int_val_ds, epochs=200, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/200
16625/16625 [==============================] - 435s 26ms/step - loss: 0.2266 - accuracy: 0.9072 - val_loss: 0.2129 - val_accuracy: 0.9165
Epoch 2/200
16625/16625 [==============================] - 442s 27ms/step - loss: 0.2024 - accuracy: 0.9185 - val_loss: 0.1996 - val_accuracy: 0.9204
Epoch 3/200
16625/16625 [==============================] - 444s 27ms/step - loss: 0.1981 - accuracy: 0.9202 - val_loss: 0.1975 - val_accuracy: 0.9199
Epoch 4/200
16625/16625 [==============================] - 444s 27ms/step - loss: 0.1950 - accuracy: 0.9217 - val_loss: 0.1928 - val_accuracy: 0.9231
Epoch 5/200
16625/16625 [==============================] - 448s 27ms/step - loss: 0.1935 - accuracy: 0.9226 - val_loss: 0.1928 - val_accuracy: 0.9231
Epoch 6/200
16625/16625 [==============================] - 447s 27ms/step - loss: 0.1914 - accuracy: 0.9232 - val_loss: 0.1917 - val_accuracy: 0.9241
Epoch 7/200
16625/16625 [==============================] - 446s 27ms/step - loss: 0.1912 - accuracy:

Without positional encoding, we seem to get a test accuracy of 92.5% which is the highest test accuracy I have ever gotten in any NLP model ever. 

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="SparseCategoricalCrossentropy",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=200, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        781312    
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

With 3000 tokens and positional embedding, I got a test accuracy of 92.5%.

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model.compile(optimizer="rmsprop",
              loss="SparseCategoricalCrossentropy",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
]
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        269312    
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

Based on 1000 word vocabulary, the test accuracy reduces from 92.4% to 90.5% which is still respectable.